<a href="https://colab.research.google.com/github/ladyj-dev/Wine_All_You_Want/blob/main/pdwine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Filtering Data

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.1'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:15 http

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkFunctions").getOrCreate()

In [3]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/dataviz-curriculum/day_1/wine.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("wine.csv"), sep=",", header=True)

# Show DataFrame
df.show()

+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|country|         description|         designation|points|price|          province|            region_1|         region_2|           variety|              winery|
+-------+--------------------+--------------------+------+-----+------------------+--------------------+-----------------+------------------+--------------------+
|     US|This tremendous 1...|   Martha's Vineyard|    96|  235|        California|         Napa Valley|             Napa|Cabernet Sauvignon|               Heitz|
|  Spain|Ripe aromas of fi...|Carodorum Selecci...|    96|  110|    Northern Spain|                Toro|             null|     Tinta de Toro|Bodega Carmen Rod...|
|     US|Mac Watson honors...|Special Selected ...|    96|   90|        California|      Knights Valley|           Sonoma|   Sauvignon Blanc|            Macauley|
|     US|This spent 20

In [4]:
import pandas as pd
pandas_df = df.toPandas()
pandas_df

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selecci�_n Especial Reserva,96,110,Northern Spain,Toro,None,Tinta de Toro,Bodega Carmen Rodr�_guez
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,France,"This is the top wine from La B̩gude, named aft...",La Br��lade,95,66,Provence,Bandol,None,Provence red blend,Domaine de la B̩gude
...,...,...,...,...,...,...,...,...,...,...
150930,Italy,Many people feel Fiano represents southern Ita...,None,91,20,Southern Italy,Fiano di Avellino,None,White Blend,Feudi di San Gregorio
150931,France,"Offers an intriguing nose with ginger, lime an...",Cuv̩e Prestige,91,27,Champagne,Champagne,None,Champagne Blend,H.Germain
150932,Italy,This classic example comes from a cru vineyard...,Terre di Dora,91,20,Southern Italy,Fiano di Avellino,None,White Blend,Terredora
150933,France,"A perfect salmon shade, with scents of peaches...",Grand Brut Ros̩,90,52,Champagne,Champagne,None,Champagne Blend,Gosset


began my analysis

In [5]:
wine_df = pandas_df.copy()
wine_df

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selecci�_n Especial Reserva,96,110,Northern Spain,Toro,None,Tinta de Toro,Bodega Carmen Rodr�_guez
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,France,"This is the top wine from La B̩gude, named aft...",La Br��lade,95,66,Provence,Bandol,None,Provence red blend,Domaine de la B̩gude
...,...,...,...,...,...,...,...,...,...,...
150930,Italy,Many people feel Fiano represents southern Ita...,None,91,20,Southern Italy,Fiano di Avellino,None,White Blend,Feudi di San Gregorio
150931,France,"Offers an intriguing nose with ginger, lime an...",Cuv̩e Prestige,91,27,Champagne,Champagne,None,Champagne Blend,H.Germain
150932,Italy,This classic example comes from a cru vineyard...,Terre di Dora,91,20,Southern Italy,Fiano di Avellino,None,White Blend,Terredora
150933,France,"A perfect salmon shade, with scents of peaches...",Grand Brut Ros̩,90,52,Champagne,Champagne,None,Champagne Blend,Gosset


In [6]:
wine_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150935 entries, 0 to 150934
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   country      150930 non-null  object
 1   description  150935 non-null  object
 2   designation  105201 non-null  object
 3   points       150929 non-null  object
 4   price        137238 non-null  object
 5   province     150924 non-null  object
 6   region_1     125877 non-null  object
 7   region_2     60971 non-null   object
 8   variety      150929 non-null  object
 9   winery       150922 non-null  object
dtypes: object(10)
memory usage: 11.5+ MB
